<a href="https://colab.research.google.com/github/dongyeon2604-png/ManUtdAnalysis/blob/main/%EC%98%81%EC%96%B4_%EA%B0%90%EC%84%B1%EC%A0%90%EC%88%98(RoBERTa).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from transformers import pipeline
from tqdm import tqdm
from google.colab import files
import io

# ==============================================================================
# 1. AI 감성분석 모델 로드 (인터넷에서 자동 다운로드)
# ==============================================================================
print("최신 AI 모델(RoBERTa)을 로드 중입니다... (약 30초 소요)")
# 0: Negative(부정), 1: Neutral(중립), 2: Positive(긍정)
sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment-latest",
    tokenizer="cardiffnlp/twitter-roberta-base-sentiment-latest",
    device=-1 # GPU가 있다면 0, 없으면 -1 (CPU)
)
print("모델 준비 완료!\n")

# ==============================================================================
# 2. 파일 업로드
# ==============================================================================
print("▼ 분석할 엑셀/CSV 파일들을 모두 선택해서 업로드해주세요.")
uploaded = files.upload()

if not uploaded:
    print("파일이 업로드되지 않았습니다.")
else:
    print("\nAI가 데이터 분석을 시작합니다...\n")
    results = []

    # AI 모델 분석 함수
    def analyze_with_ai(text):
        try:
            # 텍스트가 너무 짧거나(1자 이하) 비어있으면 패스
            if not isinstance(text, str) or len(text) < 2:
                return 0

            # 모델 예측 (최대 길이 512 토큰 제한)
            result = sentiment_pipeline(text, truncation=True, max_length=512)[0]
            label = result['label']

            # 점수 변환 (부정: -100점, 중립: 0점, 긍정: +100점)
            if label == 'negative': return -100
            elif label == 'positive': return 100
            else: return 0 # neutral
        except:
            return 0

    # ==============================================================================
    # 3. 파일별 분석 실행
    # ==============================================================================
    for filename in uploaded.keys():
        try:
            # 1) 감독/시기 정보 추출
            manager = "Unknown"
            if "모예스" in filename or "Moyes" in filename: manager = "Moyes"
            elif "무리뉴" in filename or "Mourinho" in filename: manager = "Mourinho"
            elif "반할" in filename or "Van" in filename: manager = "Van_Gaal"
            elif "솔샤르" in filename or "Solskjaer" in filename: manager = "Solskjaer"
            elif "텐하흐" in filename or "Ten" in filename: manager = "Ten_Hag"

            period = "Unknown"
            if "부임" in filename or "Start" in filename or "기대" in filename: period = "Start"
            elif "경질" in filename or "End" in filename: period = "End"

            # 2) 파일 읽기
            if filename.endswith('.xlsx'):
                df = pd.read_excel(io.BytesIO(uploaded[filename]), engine='openpyxl')
            else:
                df = pd.read_csv(io.BytesIO(uploaded[filename]))

            # 3) 텍스트 컬럼 찾기 (정규화된 토큰보다는 '원문'이 AI에게 더 좋습니다)
            # 원문 컬럼이 없다면 마지막 컬럼 사용
            target_col = df.columns[1] if len(df.columns) > 1 else df.columns[-1]

            # 4) AI 분석 실행 (오래 걸릴 수 있어 tqdm으로 진행률 표시)
            print(f"[{manager} - {period}] 분석 중...")

            # 빠른 테스트를 위해 데이터가 너무 많으면 500개만 샘플링 (전체 하려면 .head(500) 제거)
            sample_texts = df[target_col].dropna().astype(str).tolist()

            # 실제 스코어링
            scores = []
            # 진행상황을 보기 위해 tqdm 사용
            for text in tqdm(sample_texts):
                scores.append(analyze_with_ai(text))

            avg_score = np.mean(scores)
            results.append({'Manager': manager, 'Period': period, 'Score': avg_score})
            print(f" -> 평균 점수: {avg_score:.2f}\n")

        except Exception as e:
            print(f" -> [오류] {filename}: {e}")

    # ==============================================================================
    # 4. 최종 순위표 출력
    # ==============================================================================
    if results:
        result_df = pd.DataFrame(results)
        pivot_df = result_df.pivot_table(index='Manager', columns='Period', values='Score', aggfunc='mean')

        # 낙폭(실망감) 계산
        if 'Start' in pivot_df.columns and 'End' in pivot_df.columns:
            pivot_df['Disappointment'] = pivot_df['Start'] - pivot_df['End']
        else:
            pivot_df['Disappointment'] = 0

        pivot_df = pivot_df.round(2)

        print("\n" + "="*60)
        print("🤖 [AI 모델 분석 결과] 맨유 감독 여론 평가 순위")
        print("="*60)
        print("* 점수 범위: -100 (완전 부정) ~ +100 (완전 긍정)")

        if 'Start' in pivot_df.columns:
            print("\n1. 📈 기대감(부임) 순위")
            print(pivot_df.sort_values(by='Start', ascending=False)['Start'])

        if 'End' in pivot_df.columns:
            print("\n2. 📉 여론(경질) 순위")
            print(pivot_df.sort_values(by='End', ascending=True)['End'])

        if 'Disappointment' in pivot_df.columns:
            print("\n3. 💔 실망감(낙폭) 순위 (높을수록 기대 대비 실망 큼)")
            print(pivot_df.sort_values(by='Disappointment', ascending=False)['Disappointment'])

        pivot_df.to_csv("Final_AI_Ranking.csv")
        print("\n-> 결과가 'Final_AI_Ranking.csv'로 저장되었습니다.")

최신 AI 모델(RoBERTa)을 로드 중입니다... (약 30초 소요)


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


모델 준비 완료!

▼ 분석할 엑셀/CSV 파일들을 모두 선택해서 업로드해주세요.


Saving Norm_Clean_분석결과_모예스 경질 (1).xlsx to Norm_Clean_분석결과_모예스 경질 (1).xlsx
Saving Norm_Clean_분석결과_모예스 기대 (1).xlsx to Norm_Clean_분석결과_모예스 기대 (1).xlsx
Saving Norm_Clean_분석결과_무리뉴 경질전 (1).xlsx to Norm_Clean_분석결과_무리뉴 경질전 (1).xlsx
Saving Norm_Clean_분석결과_무리뉴 기대 (1).xlsx to Norm_Clean_분석결과_무리뉴 기대 (1).xlsx
Saving Norm_Clean_분석결과_반할 경질전 (1).xlsx to Norm_Clean_분석결과_반할 경질전 (1) (1).xlsx
Saving Norm_Clean_분석결과_반할 기대 (1).xlsx to Norm_Clean_분석결과_반할 기대 (1).xlsx
Saving Norm_Clean_분석결과_솔샤르 경질전 (1).xlsx to Norm_Clean_분석결과_솔샤르 경질전 (1).xlsx
Saving Norm_Clean_분석결과_솔샤르 기대 (1).xlsx to Norm_Clean_분석결과_솔샤르 기대 (1).xlsx
Saving Norm_Clean_분석결과_텐하흐 경질전 (1).xlsx to Norm_Clean_분석결과_텐하흐 경질전 (1).xlsx
Saving Norm_Clean_분석결과_텐하흐 기대 (1).xlsx to Norm_Clean_분석결과_텐하흐 기대 (1).xlsx

AI가 데이터 분석을 시작합니다...

[Moyes - End] 분석 중...


100%|██████████| 9/9 [00:04<00:00,  2.14it/s]


...

 -> 평균 점수: 30.00

[Ten_Hag - End] 분석 중...


100%|██████████| 10/10 [00:10<00:00,  1.06s/it]


 -> 평균 점수: -20.00

[Ten_Hag - Start] 분석 중...


100%|██████████| 10/10 [00:08<00:00,  1.22it/s]

 -> 평균 점수: 20.00


🤖 [AI 모델 분석 결과] 맨유 감독 여론 평가 순위
* 점수 범위: -100 (완전 부정) ~ +100 (완전 긍정)

1. 📈 기대감(부임) 순위
Manager
Mourinho     60.0
Moyes        50.0
Solskjaer    30.0
Ten_Hag      20.0
Van_Gaal    -10.0
Name: Start, dtype: float64

2. 📉 여론(경질) 순위
Manager
Moyes       -55.56
Solskjaer   -50.00
Ten_Hag     -20.00
Van_Gaal    -10.00
Mourinho      0.00
Name: End, dtype: float64

3. 💔 실망감(낙폭) 순위 (높을수록 기대 대비 실망 큼)
Manager
Moyes        105.56
Solskjaer     80.00
Mourinho      60.00
Ten_Hag       40.00
Van_Gaal       0.00
Name: Disappointment, dtype: float64

-> 결과가 'Final_AI_Ranking.csv'로 저장되었습니다.
